<a href="https://colab.research.google.com/github/stormliucong/llm-junkyard/blob/main/Training_a_LLM_with_Shakespear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/Colab Notebooks/gpt2"
import os
if not os.path.exists(path):
    os.makedirs(path)
%cd $path

## Structured roadmap to modern LLM/generative AI:

### Phase 1: Advanced Architectures (4-6 weeks)

**Modern Transformer Variants**
- Implement RoPE (Rotary Position Embedding) from scratch - used in LLaMA, PaLM
- Build RMSNorm instead of LayerNorm - more stable training
- Code SwiGLU activation function - better than ReLU/GELU
- Practice: Modify your GPT-2 to use these components

**Attention Mechanisms**
- Implement Multi-Query Attention (MQA) and Grouped-Query Attention (GQA)
- Build FlashAttention-style memory-efficient attention
- Add attention variants: Sliding window, sparse attention patterns
- Practice: Compare memory usage and speed vs standard attention

**Advanced Architectures**
- Study and implement key components of LLaMA architecture
- Build a simple Mixture of Experts (MoE) layer
- Understand and code basic retrieval-augmented generation

### Phase 2: Scaling and Efficiency (4-5 weeks)

**Distributed Training**
- Learn PyTorch DDP (DistributedDataParallel)
- Implement gradient accumulation and mixed precision training
- Study DeepSpeed ZeRO stages - implement ZeRO-1
- Practice: Scale your model training across multiple GPUs

**Memory Optimization**
- Implement gradient checkpointing
- Study and code model sharding techniques
- Learn about activation recomputation
- Practice: Train larger models with limited memory

**Inference Optimization**
- Implement KV-cache for faster autoregressive generation
- Build basic quantization (INT8, INT4)
- Code speculative decoding with a smaller draft model
- Study ONNX conversion and TensorRT optimization

### Phase 3: Training Methodologies (3-4 weeks)

**Advanced Training Techniques**
- Implement curriculum learning and data scheduling
- Study and code basic RLHF pipeline with reward models
- Learn about instruction tuning and chat formatting
- Practice: Fine-tune a model for instruction following

**Safety and Alignment**
- Implement Constitutional AI training loop
- Build rejection sampling and best-of-N training
- Study red-teaming and safety evaluation methods
- Create your own harmlessness dataset and training

### Phase 4: Multimodal and Tool Use (4-5 weeks)

**Vision-Language Models**
- Study CLIP architecture and implement vision encoder
- Build simple vision-language connector (linear projection)
- Implement basic VQA (Visual Question Answering) model
- Practice: Train on image-text pairs

**Tool Integration**
- Implement function calling and tool use training
- Build code execution environment integration
- Study agent frameworks and reasoning loops
- Create simple calculator/search tool integration

**Advanced Multimodal**
- Study recent architectures like LLaVA, InstructBLIP
- Implement video understanding basics
- Explore audio integration (speech recognition/generation)

### Phase 5: Production and Research (3-4 weeks)

**Production Systems**
- Learn vLLM for high-throughput serving
- Implement model caching and batching strategies
- Study continuous batching and request scheduling
- Build API endpoints with proper error handling

**Research Skills**
- Reproduce recent paper (choose from Anthropic, OpenAI, Google)
- Implement novel architectural component
- Design and run ablation studies
- Learn experiment tracking with Weights & Biases

### Practical Projects Throughout

**Project 1 (Phase 1-2):** Build a LLaMA-style model from scratch, train on a domain-specific dataset (legal, medical, code)

**Project 2 (Phase 3):** Create an instruction-tuned chatbot with safety measures, deploy it with a web interface

**Project 3 (Phase 4):** Build a multimodal model that can answer questions about images and execute code

**Project 4 (Phase 5):** Research project - implement and evaluate a novel technique from recent papers

### Essential Resources

**Technical Papers to Study:**
- "Attention Is All You Need" (review)
- LLaMA, LLaMA 2 papers
- InstructGPT, Constitutional AI
- Flamingo, LLaVA papers
- Recent Anthropic, OpenAI research

**Key Repositories:**
- nanoGPT (Karpathy) - study the clean implementations
- Transformers library source code
- LLaMA implementations (multiple on GitHub)
- vLLM, text-generation-inference

**Practical Skills:**
- Master Weights & Biases for experiment tracking
- Learn Hugging Face ecosystem deeply
- Get comfortable with cloud platforms (AWS, GCP)
- Study MLOps practices for model deployment

### Success Metrics

By the end, you should be able to:
- Train instruction-following models from scratch
- Implement and evaluate safety measures
- Build multimodal applications
- Deploy models efficiently in production
- Read and implement recent research papers
- Contribute to open-source LLM projects

In [ ]:
%pip install nbstripout
# in terminal nbstripout Training-a-LLM-with-Shakespear.ipynb

In [ ]:
%pip install torch

# Training of GPT2 style LLM

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
import os
# import tqdm
from tqdm.notebook import tqdm
import numpy as np
import requests
from torch.utils.data import Dataset, DataLoader
import tiktoken
import random
import string

## GPT2 Model

In [ ]:
# GPT2 Model
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, heads, max_seq, rope=True):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.heads = heads
        self.head_dim = d_model // heads
        assert self.head_dim * heads == d_model, "d_model must be divisible by heads"
        self.query_linear = nn.Linear(d_model, d_model)
        self.key_linear = nn.Linear(d_model, d_model)
        self.value_linear = nn.Linear(d_model, d_model)
        self.output_linear = nn.Linear(d_model, d_model)
        if rope:
            # add rotary position embedding on qk
            self.rotary_embedding = RotaryPositionalEmbedding(self.head_dim, max_seq)
        else:
            self.rotary_embedding = None

    def forward(self, query, key, value, mask=None):
        # b, n, s, d
        batch_size = query.size(0)
        # linear
        query = self.query_linear(query)
        key = self.key_linear(key)
        value = self.value_linear(value)
        # split - allow computing for n heads at the same time.
        query = query.view(batch_size, -1, self.heads, self.head_dim).transpose(1, 2) # b,n,s,d
        key = key.view(batch_size, -1, self.heads, self.head_dim).transpose(1, 2)

        # optional
        # RoPE for positional encoding
        if self.rotary_embedding is not None:
          query = self.rotary_embedding(query)
          key = self.rotary_embedding(key)
        value = value.view(batch_size, -1, self.heads, self.head_dim).transpose(1, 2)
        # attention
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(self.head_dim) # keep variant stable
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -float('inf'))
        else:
            # for decoder-style, add tril mask by default in both training and generating
            scores = scores.masked_fill(torch.tril(torch.ones_like(scores, device=query.device)) == 0, -1e9)
        # softmax
        attention = F.softmax(scores, dim=-1) # b, n, s, s
        # output - use contiguous to sort RAM to make view faster.
        output = torch.matmul(attention, value).transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        # linear projection
        return self.output_linear(output)

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(FeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.linear2(self.dropout(F.relu(self.linear1(x))))


class TransformerBlock(nn.Module):
    def __init__(self, d_model, heads, d_ff, max_seq, dropout=0.1, rope=False):
        super(TransformerBlock, self).__init__()
        self.attention = MultiHeadAttention(d_model, heads, max_seq, rope)
        # self.feed_forward = FeedForward(d_model, d_ff, dropout)
        self.feed_forward = SwiGLUFFN(d_model, d_ff, dropout) # use SwiGLU for gated activation.
        # self.norm1 = nn.LayerNorm(d_model)
        # self.norm2 = nn.LayerNorm(d_model)
        self.norm1 = nn.RMSNorm(d_model) # switch to RMSNorm(d_model) for stability
        self.norm2 = nn.RMSNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):

        # Use pre-norm
        # input -> norm -> attention -> residue -> norm -> ffn -> residue -> output
        # attention
        residue = x
        x = self.norm1(x)
        x = self.attention(x, x, x, mask)
        x = self.dropout(x)
        x = x + residue
        # feed forward
        residue = x
        x = self.norm2(x)
        x = self.feed_forward(x)
        x = self.dropout(x)
        x = x + residue
        return x

class GPT2(nn.Module):
    def __init__(self, vocab_size, d_model, max_seq, n_layers, heads, d_ff, dropout=0.1,rope=False):
        super(GPT2, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.position_embedding = nn.Embedding(max_seq, d_model)
        self.transformer_blocks = nn.ModuleList([TransformerBlock(d_model, heads, d_ff, max_seq, dropout, rope) for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, vocab_size) # final project layer
        self.apply(self._init_weights)
        self.d_model = d_model


    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, nn.Linear) and module.bias is not None:
                module.bias.data.zero_()

    def forward(self, x, mask=None):
        # if mask is None, casual mask will added automatically.
        token_emb = self.embedding(x)
        pos_emb = self.position_embedding(torch.arange(x.size(1), device=x.device))
        x = token_emb + pos_emb
        for transformer in self.transformer_blocks:
            x = transformer(x, mask)
        x = self.fc(x)
        return x

### RoPE - Rotary Position Embedding
- add position into q and k.
- In order to have dot product of q_i, k_j reflect relative position i-j, rotate original by θ_i,
- treate x as complext number x = a+bi, rotate θ gives a (cosθ+sinθ) + b (cosθ-sinθ)i. split x into [x1, x2], after rotate [x1cos-x2sin, x1sin+x2cos]
- add frequencies for different dimension. f = base^-2d_i/d_model

In [ ]:
class RotaryPositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_seq, base=10000):
        super(RotaryPositionalEmbedding, self).__init__()
        self.d_model = d_model # this should be d_model // n_headers
        self.max_seq = max_seq
        self.base = base
        # register buffer
        self.register_buffer('frequencies', self._get_frequencies())
        cos, sin = self._get_cos_sin(seq_len=self.max_seq)
        self.register_buffer('cos', cos)
        self.register_buffer('sin', sin)

    def _get_frequencies(self):
        # f = base^-2di/d_model
        assert self.d_model % 2 == 0, "d_model must be divisible by 2"
        frequencies = 1.0 / (self.base ** (torch.arange(0, self.d_model, 2).float() / self.d_model)) # d_model/2
        return frequencies

    def _get_cos_sin(self, seq_len=None, device=None):
        pos_i = torch.arange(seq_len).unsqueeze(1) # s, 1
        if device is not None:
          pos_i = pos_i.to(device)
        freqs = self.frequencies.unsqueeze(0) # 1, d/2
        args = pos_i * freqs # s, d/2
        cos = torch.cos(args).unsqueeze(0) # 1, s, d/2
        sin = torch.sin(args).unsqueeze(0) # 1, s, d/2
        return cos, sin

    def _apply_rope(self, x, cos, sin):
        # split x into x1, x2
        x1, x2 = x.chunk(2, dim=-1) # b, s, d/2
        assert x1.size() == x2.size()
        # rotate x1, x2
        x1 = x1 * cos - x2 * sin # b, s, d/2
        x2 = x1 * sin + x2 * cos # b, s, d/2
        # concatenate
        return torch.cat([x1, x2], dim=-1) # b, s, d

    def forward(self, x):
        if x.size(2) == self.max_seq:
            con, sin = self.cos, self.sin
        else:
            cos, sin = self._get_cos_sin(seq_len=x.size(2), device=x.device)
        return self._apply_rope(x, con, sin)


### RMSNorm
- LayerNorm: (x-μ)/σ. Shape of μ [batch_size, seq_length, 1].
- weighted parameters: α * (x-μ)/σ + β. σ can be zero, catestrophic cancellation, lead to numeric instability.
- RMSNorm: x/RMS(x): γ * x/RMS(x). Easy computing and avoid precision loss; also better gradient flow, each one is independent not relying on μ.
- LayerNorm(x) = LayerNorm(x + c1). absolute position information is not captured. position [1,2,3] will have the same as position [10,20,30].
- μ is not your friends empirically vulnerable to outliers.
- BatchNorm: (x-μ)/σ. Shape of μ [1, 1, d_model]. Not good for LLM b/c batch variability can be large and different between training and testing. Even tricky with flexible input seq_len


In [ ]:
class RMSNorm(nn.Module):
    def __init__(self, d_model, eps=1e-5):
        super(RMSNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(d_model))
        self.eps = eps
    def forward(self, x):
        return self.gamma * x / (torch.norm(x, dim=-1, keepdim=True) + self.eps)

# Pytorch has RMSNorm nn.RMSNorm(x)

### SwiGLU activation
- Similar to ReLU but with learned threshold (not like zero in ReLU)
- Better gradient flow (allow non-zero gradient for negatives; avoid dieing ReLU or die neurons)
- dead neurons: relu(x) = 0 receive 0 gradient, and stay 0 forever.
- 2010s thinking: "Dead neurons are bad, but sparsity is good"
- 2020s thinking: "We want sparsity but not permenant dead"

In [ ]:
class SwiGLUFFN(nn.Module):
    # SiLU(xW) * (xV)
    def __init__(self, d_model, d_ff=None, dropout=0.1):
        super(SwiGLUFFN, self).__init__()
        if d_ff is None:
          d_ff = int(d_model * 8/3)
        self.gate_proj = nn.Linear(d_model, d_ff, bias=False)
        self.up_proj= nn.Linear(d_model, d_ff, bias=False)
        self.down_proj = nn.Linear(d_ff, d_model, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        gate = F.silu(self.gate_proj(x))
        up = self.up_proj(x)
        hidden = gate * up
        down = self.down_proj(self.dropout(hidden))
        return down


### Multiple Query Attention (MQA)
- shared k,v matrix across all headers
- much faster, lower performance reduce


In [ ]:
class MultiQueryAttention(nn.Module):
    def __init__(self, d_model, heads, max_seq, rope=True):
        super(MultiQueryAttention, self).__init__()
        self.d_model = d_model
        self.heads = heads
        self.head_dim = d_model // heads
        assert self.head_dim * heads == d_model, "d_model must be divisible by heads"
        self.wq = nn.Linear(d_model, d_model)
        self.wk = nn.Linear(d_model, self.head_dim) # only head_dim, shared across headers
        self.wv = nn.Linear(d_model, self.head_dim) # only head_dim, shared across headers
        self.output_linear = nn.Linear(d_model, d_model)
        if rope:
            self.rotary_embedding = RotaryPositionalEmbedding(self.head_dim, max_seq)
        else:
            self.rotary_embedding = None

    def forward(self, query, key, value, mask=None):
        # b, n, s, d
        batch_size = query.size(0)
        # linear
        query = self.wq(query) # b,s,d*n
        key = self.wk(key) # b,s,d
        value = self.wv(value) # b,s,d
        # split
        query = query.view(batch_size, -1, self.heads, self.head_dim).transpose(1, 2) # b,n,s,d
        key = key.unsqueeze(1).expand(-1, query.size(1), -1, -1)
        value = value.unsqueeze(1).expand(-1, query.size(1), -1, -1)
        # optional
        # RoPE for positional encoding
        if self.rotary_embedding is not None:
            query = self.rotary_embedding(query)
            key = self.rotary_embedding(key)
        # attention
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(self.head_dim)

        # mask
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        else:
            scores = scores.masked_fill(torch.tril(torch.ones_like(scores, device=query.device)) == 0, -1e9)
        # softmax
        attention = F.softmax(scores, dim=-1) # b, n, s, s
        # output
        output = torch.matmul(attention, value).transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        # linear projection
        return self.output_linear(output)

### Grouped Query Attention
- Compromise between MQA and MHA.
- Group some headers to share K, V

In [ ]:
class GroupedQueryAttention(nn.Module):
    def __init__(self, d_model, heads, kv_groups, max_seq, rope=True):
        super(GroupedQueryAttention, self).__init__()
        self.d_model = d_model
        self.heads = heads
        self.kv_groups = kv_groups
        self.heads_per_group = heads // kv_groups
        assert self.heads_per_group * kv_groups == heads, "heads must be divisible by kv_groups"
        self.head_dim = d_model // heads
        assert self.head_dim * heads == d_model, "d_model must be divisible by heads"
        self.group_dim = self.head_dim * self.kv_groups
        self.wq = nn.Linear(d_model, d_model)
        self.wk = nn.Linear(d_model, self.group_dim)
        self.wv = nn.Linear(d_model, self.group_dim)
        self.output_linear = nn.Linear(d_model, d_model)
        if rope:
            self.rotary_embedding = RotaryPositionalEmbedding(self.head_dim, max_seq)
        else:
            self.rotary_embedding = None

    def forward(self, query, key, value, mask=None):
        # b, n, s, d
        batch_size = query.size(0)
        # linear
        query = self.wq(query) # b,s,d*n
        key = self.wk(key) # b,s,d*m
        value = self.wv(value) # b,s,d*m
        # split
        query = query.view(batch_size, -1, self.heads, self.head_dim).transpose(1, 2) # b,n,s,d
        key = key.view(batch_size, -1, self.kv_groups, self.head_dim).transpose(1, 2).repeat_interleave(self.heads_per_group, dim=1)
        value = value.view(batch_size, -1, self.kv_groups, self.head_dim).transpose(1, 2).repeat_interleave(self.heads_per_group, dim=1)
        # rope
        if self.rotary_embedding is not None:
            query = self.rotary_embedding(query)
            key = self.rotary_embedding(key)
        # score
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(self.head_dim)
        # mask
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        else:
            scores = scores.masked_fill(torch.tril(torch.ones_like(scores, device=query.device)) == 0, -1e9)
        # softmax
        attention = F.softmax(scores, dim=-1) # b, n, s, s
        # output
        output = torch.matmul(attention, value).transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        # linear projection
        return self.output_linear(output)


### FlashAttention
- The key concept of FlashAttention is there is a memory movement cost (which is the bottleneck) between  fast RAM or L1/L2 Caches (SRAM, 100TB/s) and global RAM in CUDA GPU (HBM/DRAM, 2TB/s). This is similar to I/O cost.
- Everytime, pytorch line (or more accurately, a cuda kernel) will materialize (similar to writing out to disk) the variable in the global RAM.
- However, we don't have to do that, if we calculate everything in HRAM, and store the calculated variables in the HRAM for later kernel operation
- That will require us to break down the large matrix calculation into blocked matrix calculation (b/c the HRAM is smaller than global RAM)
- But you can't really implement this by change the pytorch code b/c pytorch will still materialize the blocked matrix calculation results (determined by its kernel function).
- The easiest you can do is to use `scaled_dot_product_attention`, which do blocked kernel calculation under the hood.
- You can also use Triton, which is a python way to implement cuda kernels. see [this example](https://arunjitha.medium.com/simplifying-cuda-kernels-with-triton-a-pythonic-approach-to-gpu-programming-79bb7121e974)

In [ ]:
from torch.nn.functional import scaled_dot_product_attention
from torch.nn.attention import SDPBackend, sdpa_kernel

class FlashAttention(nn.Module):
    def __init__(self, d_model, heads, max_seq, rope=True):
        super(FlashAttention, self).__init__()
        self.d_model = d_model
        self.heads = heads
        self.head_dim = d_model // heads
        assert self.head_dim * heads == d_model, "d_model must be divisible by heads"
        self.query_linear = nn.Linear(d_model, d_model)
        self.key_linear = nn.Linear(d_model, d_model)
        self.value_linear = nn.Linear(d_model, d_model)
        self.output_linear = nn.Linear(d_model, d_model)
        if rope:
            self.rotary_embedding = RotaryPositionalEmbedding(self.head_dim, max_seq)
        else:
            self.rotary_embedding = None
    def forward(self, query, key, value, mask=None):
        # b, n, s, d
        batch_size = query.size(0)
        # linear
        query = self.query_linear(query)
        key = self.key_linear(key)
        value = self.value_linear(value)
        # split
        query = query.view(batch_size, -1, self.heads, self.head_dim).transpose(1, 2) # b,n,s,d
        key = key.view(batch_size, -1, self.heads, self.head_dim).transpose(1, 2)
        value = value.view(batch_size, -1, self.heads, self.head_dim).transpose(1, 2)
        # optional
        # RoPE for positional encoding
        if self.rotary_embedding is not None:
            query = self.rotary_embedding(query)
            key = self.rotary_embedding(key)
        # attention with flash_attention implemented under the hood
        with sdpa_kernel(SDPBackend.FLASH_ATTENTION):
            scores = scaled_dot_product_attention(query, key, value, attn_mask=mask, enable_gqa=False)
        # output
        return self.output_linear(scores)


### Attention with Sliding window and Sparse Attention
- mainly used in long-context scenario
- sparse attention usually include global attention + local attention
- global attention can be random attention; first_n token attention and stride-based attention;
- the implementation is to create various mask where mask[i,j] = 1 means i will attend to j.
- different attention can be combined as hierachical ways to form a bigbird mask
- there are other attention ways like hash bucket based or other row-wise and column-wise based. But the overall idea is similar.

In [ ]:
def create_local_masks(seq_len, local_window_size):
    masks = torch.zeros(seq_len, seq_len, dtype=torch.bool)
    masks.fill_diagonal_(1)
    for i in range(seq_len):
        # gpt always attend to tokens before i
        start = max(0, i - local_window_size)
        end = i
        masks[i, start:end] = True
    return masks.unsqueeze(0).unsqueeze(0)

def create_global_first_n_masks(seq_len, first_n):
    masks = torch.zeros(seq_len, seq_len, dtype=torch.bool)
    masks.fill_diagonal_(1)
    for i in range(seq_len):
        start = 0
        end = min(first_n, i)
        masks[i, start:end] = True
    return masks.unsqueeze(0).unsqueeze(0)

def create_global_stride_masks(seq_len, stride):
    masks = torch.zeros(seq_len, seq_len, dtype=torch.bool)
    masks.fill_diagonal_(1)
    for i in range(seq_len):
        start = 0
        end = i
        masks[i, start:end:stride] = True
    return masks.unsqueeze(0).unsqueeze(0)

def create_global_random_masks(seq_len, prob=0.1, device='cpu', seed=42):
    # fix seed to ensure training and generating are same
    generator = torch.Generator(device=device)
    generator.manual_seed(seed)
    masks = torch.zeros(seq_len, seq_len, dtype=torch.bool)
    masks.fill_diagonal_(1)
    for i in range(1, seq_len):
        random_vals = torch.rand(i, generator=generator, device=device)
        masks[i, :i] = random_vals < prob
    return masks.unsqueeze(0).unsqueeze(0)

### Weight Tieing
- The embedding layer and last projection layer share parameters.
- Intuitively it make sense, it also saves the paremeters (d_model * vocabulary_size)
- Empirically, Llama implemented in that way.
- Two ways to implement, in GPT style, same parameter will receive gradient from two paths embedding and final fc layer; In llama, simply multiple embedding weight to get logits, no gradient flow in the last layer.

In [ ]:
class GPTwithWeightTier(GPT2):
    def __init__(self, vocab_size, d_model, max_seq, n_layers, heads, d_ff, dropout=0.1, rope=False, weight_tie=False):
        super(GPTwithWeightTier, self).__init__(vocab_size, d_model, max_seq, n_layers, heads, d_ff, dropout, rope)
        if weight_tie:
            assert self.embedding is not None, "embedding layer must be defined"
            self.fc = nn.linear(d_model, vocab_size, bias=False) # it is a function
            self.fc.weight = self.embedding.weight
    def forward(self, x, mask=None):
        return super(GPTwithWeightTier, self).forward(x, mask)

### Mixture of Experts (MoE)
- replace the FFN after attention
- Original TransformerBlock: LayerNorm -> Attention -> Residue -> LayerNorm -> FFN -> NextBlock
- MoEBlock: LayerNorm -> Attention -> Residue -> Layernomr -> MoE -> NextBlock
- The idea is instead of using only one FFN, we want to use weighted sum of num_experts number of FFNs (could be smaller d_ff). But we don't want to calculate num_experts for each token to increase the computational burden by num_experts times. Instead, we used a gated mechanism to only activate topk based on a gate_activation function.


In [ ]:
class SingleExpertFFN(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(SingleExpertFFN, self).__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.linear2(self.dropout(F.relu(self.linear1(x))))

class GatedFFN(nn.Module):
    def __init__(self, d_model, top_k, num_experts):
        super(GatedFFN, self).__init__()
        self.gate_proj = nn.Linear(d_model, num_experts, bias=False)
        self.top_k = top_k
        self.num_experts = num_experts

    def _compute_load_balance_loss(self, gate_scores, top_k_indices):
        gate_prob = F.softmax(gate_scores, dim=-1) # b, s, e
        mean_gate_prob = gate_prob.mean(dim=[0,1]) # e
        tokens_per_experts = torch.zeros(self.num_experts, device=gate_scores.device) # e
        for i in range(self.num_experts):
            tokens_per_experts[i] = (top_k_indices == i).float().sum()
        # normalize
        total_tokens = gate_scores.size(0) * gate_scores.size(1) * self.top_k # b*s*k
        token_usage_gate_prob = tokens_per_experts / total_tokens # e, sum(e) = 1
        # compute loss - penalize deviations from uniform expert usage.
        # similar to entropy but considered both continues prob (mean_gate_prob) and cut-off prob (token_usage)
        return torch.sum(token_usage_gate_prob * mean_gate_prob) * self.num_experts

    def forward(self, x):
        gate_scores = self.gate_proj(x) # b, s, e
        top_k_scores, top_k_indices = torch.topk(gate_scores, self.top_k, dim=-1) # b, s, k
        top_k_gate_weight = F.softmax(top_k_scores, dim=-1) # b, s, k

        if self.training:
            load_balance_loss = self._compute_load_balance_loss(gate_scores, top_k_indices)

        else:
            load_balance_loss = torch.tensor(0.0, device=x.device)

        return top_k_gate_weight, top_k_indices, load_balance_loss

class MoEFFN(nn.Module):
    def __init__(self, d_model, top_k, d_ff, num_experts, dropout=0.1):
        super(MoEFFN, self).__init__()
        self.num_experts = num_experts
        self.expert_ffns = nn.ModuleList([SingleExpertFFN(d_model, d_ff, dropout) for _ in range(num_experts)])
        self.gate_activation = GatedFFN(d_model, top_k, num_experts)
        self.top_k = top_k
    def forward(self, x):
        top_k_gate_weight, top_k_indices, load_balance_loss = self.gate_activation(x) # b,s,k
        flattened_x = x.view(-1, x.size(-1)) # b*s, d -> b*s tokens, each token has own top_k gate and prob
        flattened_top_k_gate_weight = top_k_gate_weight.view(-1, top_k_gate_weight.size(-1)) # b*s, k
        flattened_top_k_indices = top_k_indices.view(-1, top_k_indices.size(-1)) # b*s, k
        weighted_expert_outputs = torch.zeros_like(flattened_x) # b*s, d
        for j in range(self.top_k):
          # get current expert idx
          top_k_expert_idx = flattened_top_k_indices[:, j] # b*s
          top_k_expert_weight = flattened_top_k_gate_weight[:, j] # b*s
          # process current j position for each token
          for i in range(self.num_experts):
              expert_mask = (top_k_expert_idx == i) # b*s
              if expert_mask.any():
                # only process expert with at least one tokens
                expert_input = flattened_x[expert_mask] # ?, d
                expert_output = self.expert_ffns[i](expert_input) # ?, d
                expert_weight = top_k_expert_weight[expert_mask].unsqueeze(-1) # ?, 1
                weighted_expert_outputs[expert_mask] += expert_weight * expert_output # ?, 1

        return weighted_expert_outputs.view(x.size(0), x.size(1), -1),  load_balance_loss



## Dataset

In [ ]:
# Dataset
class TextDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.seq = self.tokenizer.encode(data)
        self.seq_num = len(self.seq) - max_length - 1

    def __len__(self):
        return self.seq_num

    def __getitem__(self, idx):
        input = self.seq[idx:idx+self.max_length]
        target = self.seq[idx+1:idx+self.max_length+1]
        return torch.tensor(input, dtype=torch.long), torch.tensor(target, dtype=torch.long)

## Train

In [ ]:
# Train
class Trainer:
    def __init__(self, model, device, learning_rate = 1e-3, weight_decay = 0.01, warmup_steps = 1000, max_steps = 10000, gradient_accumulation_steps = 1, grad_clip = 1.0, save_dir = "./checkpoints"):
        self.model = model
        self.optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        self.criterion = nn.CrossEntropyLoss()
        self.device = device
        self.model.to(self.device)
        self.scheduler = self._get_scheduler(warmup_steps, max_steps)
        self.gradient_accumulation_steps = gradient_accumulation_steps
        self.grad_clip = grad_clip
        self.save_dir = save_dir
        self.global_step = 0
        self.best_loss = float('inf')
        os.makedirs(save_dir, exist_ok=True)

    def _get_scheduler(self, warmup_steps, max_steps):
        def lr_lambda(current_step):
            if current_step < warmup_steps:
                return float(current_step) / float(max(1, warmup_steps))
            # Cosine decay after warmup
            progress = float(current_step - warmup_steps) / float(max(1, max_steps - warmup_steps))
            return max(0.0, 0.5 * (1.0 + torch.cos(torch.pi * torch.tensor(progress))))
        return optim.lr_scheduler.LambdaLR(self.optimizer, lr_lambda)

    def train(self, train_dataloader, validate_dateloader, epochs):
        self.model.train()

        for epoch in range(epochs):
            print(f"Epoch {epoch+1}/{epochs}")
            total_loss = 0
            pb = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}")
            for batch_idx, (data, target) in enumerate(pb):
                data, target = data.to(self.device), target.to(self.device)
                self.optimizer.zero_grad()
                output = self.model(data)
                loss = self.criterion(output.view(-1, output.size(-1)), target.view(-1))
                loss = loss / self.gradient_accumulation_steps if self.gradient_accumulation_steps > 1 else loss
                loss.backward()
                if (batch_idx + 1) % self.gradient_accumulation_steps == 0:
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.grad_clip)
                    self.optimizer.step()
                    self.scheduler.step()
                    self.global_step += 1
                # update pb for loss
                total_loss += loss.item() * self.gradient_accumulation_steps
                # update pb per 100
                if batch_idx % 100 == 0:
                    pb.set_postfix(loss=f"{total_loss / (batch_idx + 1):4f}")
            # call validate
            self.validate(validate_dateloader)
        print(f"Training finished")
        return 1

    def validate(self, dataloader):
        self.model.eval()
        total_loss = 0
        pb = tqdm(dataloader, desc="Validation")
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(pb):
                data, target = data.to(self.device), target.to(self.device)
                output = self.model(data)
                loss = self.criterion(output.view(-1, output.size(-1)), target.view(-1))
                total_loss += loss.item()
                # update pb per 100
                if batch_idx % 100 == 0:
                    pb.set_postfix(loss=f"{total_loss / (batch_idx + 1):4f}")

        if (total_loss / (batch_idx + 1)) < self.best_loss:
            self.best_loss = (total_loss / (batch_idx + 1))
            self.save_checkpoint()

        return 1

    def save_checkpoint(self, filename='best_model.pt'):
        checkpoint_path = os.path.join(self.save_dir, filename)
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'global_step': self.global_step,
            'best_loss': self.best_loss
        }, checkpoint_path)
        print(f"Checkpoint saved to {checkpoint_path}")

    def load_checkpoint(self, filename='best_model.pt'):
        checkpoint_path = os.path.join(self.save_dir, filename)
        checkpoint = torch.load(checkpoint_path)
        self.model_config = checkpoint['model_config']
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.global_step = checkpoint['global_step']
        self.best_loss = checkpoint['best_loss']
        print(f"Checkpoint loaded from {checkpoint_path}")

### DDP (Distributed Data Parallel)
- across multiple nodes (x gpu per nodes)
- each node contains same dataset replica, model and environment
- node know each other through world size and master node
- each iteration update same model (model wrapped using DDP), same parameter, but different sampled dataset (from a same dataset) across nodes
- back propogation (loss) auto syncronized (all reduced) through torch.distributed
- able to process larger batch size (128 in one node with 2 gpu -> 128 * 8 with 4 nodes with 2 gpus), improve training efficiency
- only save one copy and print one loss b/c they are all syned across nodes (rank = 0)
- support both gpu (GLOO) and cpu (NCCL) training
- use either torchrun (production, better fault tolerance) or multiprocessing.spawn


In [ ]:
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.multiprocessing as mp
from torch.utils.data.distributed import DistributedSampler


class DDPTrainer:
    def __init__(self, model, world_size,, learning_rate = 1e-3, weight_decay = 0.01, warmup_steps = 1000, max_steps = 10000, gradient_accumulation_steps = 1, grad_clip = 1.0, save_dir = "./checkpoints"):
        self.model = model
        self.world_size = world_size
        self.gradient_accumulation_steps = gradient_accumulation_steps
        self.grad_clip = grad_clip
        self.save_dir = save_dir
        self.global_step = 0
        self.best_loss = float('inf')
        os.makedirs(save_dir, exist_ok=True)

    @staticmethod
    def _init_setup(rank, world_size, backend="nccl"):
        # nccl for gpu; gloo for cpu
        os.environ['MASTER_ADDR'] = 'localhost'
        os.environ['MASTER_PORT'] = '12355'
        torch.cuda.set_device(rank)
        dist.init_process_group(backend=backend, rank=rank, world_size=world_size)

    @staticmethod
    def _cleanup():
        dist.destroy_process_group()

    def _get_scheduler(self, warmup_steps, max_steps):
        def lr_lambda(current_step):
            if current_step < warmup_steps:
                return float(current_step) / float(max(1, warmup_steps))
            # Cosine decay after warmup
            progress = float(current_step - warmup_steps) / float(max(1, max_steps - warmup_steps))
            return max(0.0, 0.5 * (1.0 + torch.cos(torch.pi * torch.tensor(progress))))
        return optim.lr_scheduler.LambdaLR(self.optimizer, lr_lambda)

    def train_master(self, train_dataset, val_dataloader, epochs, batch_size):
        # master to spawn multiprocess
        mp.spawn(self.train_worker, args=(train_dataset, val_dataloader, epochs, batch_size), nprocs=self.world_size, join=True)
        return 1


    def train_worker(self, rank, train_dataset, val_dataloader, epochs, batch_size):

        # init progress group
        self._init_setup(rank, self.world_size)

        self.device = torch.device(f'cuda:{rank}')

        # init model for this worker
        self.model = self.model.to(self.device)
        model = DDP(self.model, device_ids=[rank], output_device=rank)
        model.train()

        # Initialize optimizer, scheduler, and criterion for this worker
        self.optimizer = optim.AdamW(
            model.parameters(),
            lr=self.learning_rate,
            weight_decay=self.weight_decay
        )
        self.scheduler = self._get_scheduler(self.optimizer, self.warmup_steps, self.max_steps)
        self.criterion = nn.CrossEntropyLoss()

        #  load dataset - the entire data copy is needed in each node
        train_dataset_sampler = DistributedSampler(self.train_dataset, num_replicas=self.world_size, rank=rank)
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_dataset_sampler, pin_memory=True)
        val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)


        for epoch in range(epochs):
            # Set epoch for sampler to ensure different shuffling each epoch
            train_sampler.set_epoch(epoch)
            total_loss = 0
            num_batches = 0

            if rank == 0:
                pbar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}")
            else:
                pbar = train_dataloader
            for batch_idx, (data, target) in enumerate(pbar):
                data, target = data.to(self.device, non_blocking=True), target.to(self.device, non_blocking=True)
                output = self.model(data)
                loss = self.criterion(output.view(-1, output.size(-1)), target.view(-1))
                loss = loss / self.gradient_accumulation_steps if self.gradient_accumulation_steps > 1 else loss
                loss.backward()
                if (batch_idx + 1) % self.gradient_accumulation_steps == 0:
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.grad_clip)
                    self.optimizer.step()
                    self.scheduler.step()
                    self.optimizer.zero_grad()
                    self.global_step += 1
                # update pb for loss
                total_loss += loss.item() * self.gradient_accumulation_steps
                num_batches += 1
                # Update progress bar
                if batch_idx % 100 == 0 and rank == 0: # only print in node 0.
                    average_loss = total_loss / num_batches
                    pbar.set_postfix(loss=f"{average_loss:.4f}", step=self.global_step)

            # call validate
            if rank == 0:
              self.validate(model, validate_dateloader)

        self._cleanup()
        return 1

    def validate(self, model, dataloader):
        self.model.eval()
        total_loss = 0
        number_batches = 0

        pb = tqdm(dataloader, desc="Validation")
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(pb):
                data, target = data.to(self.device), target.to(self.device)
                output = self.model(data)
                loss = self.criterion(output.view(-1, output.size(-1)), target.view(-1))
                total_loss += loss.item()
                num_batches += 1
                # update pb per 100
                if batch_idx % 100 == 0:
                    pb.set_postfix(loss=f"{total_loss / num_batches:4f}")

        avg_loss = total_loss / num_batches if num_batches > 0 else float('inf')

        if avg_loss < self.best_loss:
            self.best_loss = avg_loss
            self.save_checkpoint(model, loss)

        return 1

    def save_checkpoint(self, model, loss):
        checkpoint_path = os.path.join(self.save_dir, 'best_model.pt')
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'global_step': self.global_step,
            'best_loss': self.best_loss
            'loss': loss
        }, checkpoint_path)
        print(f"Checkpoint saved to {checkpoint_path}")

if __name__ == "__main__":
    world_size = 4
    trainer = DDPTrainer(model, world_size)
    train_dataset = TextDataset(train_data[1:10000], tokenizer, model_config['max_seq'])
    train_dataloader = DataLoader(train_dataset, batch_size=200, shuffle=True)
    epochs = 100
    trainer.train_master(train_dataset, val_dataloader, epochs)


### Tensor Parallel
- DDP is faster but the bottleneck in modern LLM is GPU RAM.
- What if the model and its parameters can not fit inside one GPU?
- We have to parallel and share models parameters across GPUs.
- Column parallel for a linear layer [int, out] -> [int, out/number_of_gpus]
- Row parallel for a linear layer [int, out] -> [int/number_of_gpus, out]
- use `dist.all_reduce`, `dist_all_gather` to communicate across devices

In [ ]:
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.multiprocessing as mp


class ColumnParallelLinear(nn.Module):
    def __init__(self, in_features, out_features, rank, world_size, gather_output=True):
        super(ShardedLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.world_size = world_size
        self.rank = rank
        self.local_out_features = out_features // world_size
        self.weight = nn.Parameter(torch.randn(in_features, self.local_out_features))
        self.bias = nn.Parameter(torch.zeros(self.local_out_features))
        self.gather_output = gather_output

    def forward(self, x):
        local_output = F.linear(x, self.weight.t(), self.bias) # b, s, d // ws
        if not self.gather_output:
            return local_output
        output_list = [torch.zeros_like(local_output) for _ in range(self.world_size)]
        dist.all_gather(output_list, local_output) # b, s, d // ws
        # cat
        output = torch.cat(output_list, dim=-1)
        return output

def train_workder(rank, world_size, in_features, out_features):
    # setup env. etc.

    device = torch.device(f'cuda:{rank}')
    model = ShardedLinear(in_features, out_features, rank, world_size).to(device)
    model.train()

    x = torch.randn(10, in_features).to(device)
    y = model(x)
    # loss update

    # clean up



# training example
# init dist setup ,etc.
in_features = 1024
out_features = 2048
rank = 0
world_size = 4
mp.spawn(train_workder, args=(world_size,in_features, out_features), nprocs=world_size, join=True)



### ZeRO (Zero Redundancy Optimizer)
- Optimizer stores parameters, gradient and states therefore cost lots of memory
- For SGD, the state is just learning rate, so it is cheap. But in AdamW, there is m (first momentum) and v (second momentum) per parameter, so the state memory is 2*parameter - very expensive.
- ZeRO is to shard state.
- FSDP - shard state, gradient, and parameters. Reduces memory mostly but adds coordination during step().
- DeepSpeed - shard state and gradient (middle ground)

In [ ]:
from torch.distribute.optim import ZeroRedundancyOptimizer

class ZeROTrainer(Trainer):
    def __init__(self, model, device, learning_rate = 1e-3, weight_decay = 0.01, warmup_steps = 1000, max_steps = 10000, gradient_accumulation_steps = 1, grad_clip = 1.0):
        super(ZeROTrainer, self).__init__(model, device, learning_rate, weight_decay, warmup_steps, max_steps, gradient_accumulation_steps, grad_clip)

    # overwrite train_worker with ZeroRedundancyOptimizer
    def train_worker(self, rank, train_dataset, val_dataloader, epochs, batch_size):
      self.optimizer = ZeroRedundancyOptimizer(self.optimizer, self.model.parameters())



### Mixed Precision
- using float16 instead of fp32 to save memory and accelerate operation
- NVIDIA tensor core can handle matrix multiplication with fp16 much faster
- add operation using fp32 to maintain numerical stability, multiplication using fp16 to make it faster
- using torch autocaster context to make it stable
- other format - fp32 (no tensor core), fp16, bf16, tf32, etc.

In [ ]:
from torch.cuda.amp import GradScaler, autocast

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

class MixedPrecisionTrainer:
    def __init__(self, model):
        self.model = model
        self.scaler = GradScaler()
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters())
        assert torch.cuda.is_available(), "CUDA is not available"
        self.device = torch.device("cuda")
        self.model.to(self.device)

    def train(self, train_dataloader, epochs):
        self.model.train()
        for epoch in range(epochs):
            print(f"Epoch {epoch+1}/{epochs}")
            total_loss = 0
            pb = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}")
            for batch_idx, (data, target) in enumerate(pb):
                data, target = data.to(self.device), target.to(self.device)
                with autocast(): # cast to fp16 for matmul operations
                    output = self.model(data)
                    loss = self.criterion(output, target)
                self.scaler.scale(loss).backward() # scale the number by a constant to avoid fp16 underflow
                self.scaler.step(self.optimizer) # unscale gradients before applying them
                self.scaler.update() # adjust scale factor
                self.optimizer.zero_grad()
                total_loss += loss.item()
                # update pb per 100
                if batch_idx % 100 == 0:
                    pb.set_postfix(loss=f"{total_loss / (batch_idx + 1):4f}")

            # validate
            self.validate(validate_dataloader)
    def validate(self, dataloader):
        self.model.eval()
        total_loss = 0
        pb = tqdm(dataloader, desc="Validation")
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(pb):
                data, target = data.to(self.device), target.to(self.device)
                with autocast(): # ops (e.g.matmul) using FP16, ops (e.g. softmax) using FP32
                    output = self.model(data)
                    loss = self.criterion(output, target)
                    total_loss += loss.item()

### Gradient Checkpoint
- During forward pass, activation/output/x of each layers will be calculated and stored, this is needed to calculate gradient in the backward. This will need lots of GPU RAM.
- Solution, we don't store all x. We only store some x in the GPU RAM, and calculate the other x (e.g. x = relu(x), only store the x passed to relu) again when it is needed in backward pass.
- trade-off between RAM and computing time
- set preserve_rng_state = True to allow same mask inside those discarded hidden layers

In [ ]:
from torch.utils.checkpoint import checkpoint

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

class BigNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, layer_num, use_gc = True):
        super(BigNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.blocks = nn.ModuleList([SimpleNN(hidden_size, hidden_size, hidden_size) for _ in range(layer_num)])
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.use_gc = use_gc

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        for block in self.blocks:
            if self.use_gc:
                # set preserve_rng_state = True if drop out in block
                x = checkpoint(block, x, use_reentrant=False, preserve_rng_state=False) # don't store the hidden layer output in simpleNN
            else:
                x = block(x)
        return x

# A minimal training example
def run_one_batch(use_gc):
    device = "cuda"
    model = Model(size=4096, depth=12, use_gc=use_gc).to(device)
    data = torch.randn(32, 4096, device=device, requires_grad=True)
    target = torch.randn(32, 4096, device=device)

    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.MSELoss()

    torch.cuda.reset_peak_memory_stats(device)

    output = model(data)
    loss = criterion(output, target)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    # check memory
    max_mem = torch.cuda.max_memory_allocated(device) / 1e6  # MB
    return max_mem

max_mem_gc = run_one_batch(True)
max_mem_no_gc = run_one_batch(False)
print(f"Max memory with gradient checkpointing: {max_mem_gc:.2f} MB")
print(f"Max memory without gradient checkpointing: {max_mem_no_gc:.2f} MB")

## Generate

In [ ]:
class TextGenerator:
    def __init__(self, model, tokenizer, device, max_length=100, eos_token=None):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.max_length = max_length
        self.model.to(self.device)
        self.model.eval()
        if eos_token is None:
            self.eos_token = self.tokenizer.eot_token
        else:
            self.eos_token = eos_token

    def generate(self, prompt, num_samples=1, temperature=1.0, top_k=50):
        tokens = self.tokenizer.encode(prompt)
        tokens = torch.tensor(tokens, dtype=torch.long, device=self.device).unsqueeze(0) # 0, s
        for _ in range(num_samples):
            for _ in range(self.max_length - len(tokens)):
                logits = self.model(tokens)
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)
                filtered_logits = logits.topk(top_k)[0]
                probs = F.softmax(filtered_logits, dim=-1)
                next_token = torch.multinomial(probs, num_samples=1)
                tokens = torch.cat((tokens, next_token), dim=1)
                if next_token.item() == self.eos_token:
                    break
            generated_text = self.tokenizer.decode(tokens[0].tolist())
            print(generated_text)

## Training Shakespear GPT2

In [ ]:
# define all training parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# get gpt2 tokenizer and vocab_size
tokenizer = tiktoken.get_encoding("gpt2")
vocab_size = tokenizer.n_vocab
# model parameters
model_config = {
    "vocab_size": vocab_size,
    "d_model": 128,
    "max_seq": 128,
    "n_layers": 4,
    "heads": 4,
    "d_ff": 256,
    "dropout": 0.1,
    "rope": True
}
# training parameters
batch_size=2
learning_rate = 1e-3
weight_decay = 0.01
warmup_steps = 1000
max_steps = 10000
gradient_accumulation_steps = 5
grad_clip = 1.0
epochs = 2
# train and val data
if os.path.exists("data/t8.shakespeare.txt"):
    # if the file exists, read from it
    with open("data/t8.shakespeare.txt", "r") as f:
        data = f.read()
else:
    shakespear_content_url = "https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt"
    # read the content from the URL
    data = requests.get(shakespear_content_url).text
    # save to content to local file
    if not os.path.exists("data"):
        os.makedirs("data")
    with open("data/t8.shakespeare.txt", "w") as f:
        f.write(data)
train_data, val_data = data[:int(len(data)*0.8)], data[int(len(data)*0.8):]
train_dataloader = DataLoader(TextDataset(train_data[1:10000], tokenizer, model_config['max_seq']), batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(TextDataset(val_data[1:1000], tokenizer, model_config['max_seq']), batch_size=batch_size, shuffle=True)
model = GPT2(**model_config)
# init model and trainer
trainer = Trainer(model, device, learning_rate, weight_decay, warmup_steps, max_steps, gradient_accumulation_steps, grad_clip, "checkpoints")
# load model if available
if os.path.exists("checkpoints/best_model.pt"):
    try:
      trainer.load_checkpoint()
    except Exception as e:
      print(e)
# start training
trainer.train(train_dataloader, val_dataloader, epochs)

## Inference on Trained GPT2

In [ ]:
# generate
# load bst model
model = GPT2(**model_config)
model.load_state_dict(torch.load("checkpoints/best_model.pt")['model_state_dict'])
eos_token = tokenizer.eot_token
text_generator = TextGenerator(model, tokenizer, device, max_length=100, eos_token=eos_token)
prompt = "he is a "
text_generator.generate(prompt, num_samples=1, temperature=1.0, top_k=50)